In [13]:
!python -m pip install --quiet --upgrade pip
import torch
print(f"torch version: {torch.__version__}")

torch version: 1.7.1


In [14]:
!pip install --quiet -U transformers

### Download Summarizer ### 

In [1]:
from transformers import pipeline

In [2]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [3]:
summarizer.save_pretrained("./model")

### Package the pre-trained model and upload it to S3 ###

In [23]:
import os
import boto3
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()

In [24]:
bucket = "ai-inference-env"
prefix = "summarizer"

In [21]:
!ls -rtlh ./model/

total 1.2G
-rw-rw-r-- 1 ec2-user ec2-user 1.9K Oct 19 03:14 config.json
-rw-rw-r-- 1 ec2-user ec2-user 1.2K Oct 19 03:14 tokenizer_config.json
-rw-rw-r-- 1 ec2-user ec2-user  772 Oct 19 03:14 special_tokens_map.json
-rw-rw-r-- 1 ec2-user ec2-user 1.2G Oct 19 03:14 pytorch_model.bin
-rw-rw-r-- 1 ec2-user ec2-user 780K Oct 19 03:14 vocab.json
-rw-rw-r-- 1 ec2-user ec2-user 446K Oct 19 03:14 merges.txt
-rw-rw-r-- 1 ec2-user ec2-user 1.3M Oct 19 03:14 tokenizer.json


In [22]:
!cd model && tar czvf ../model.tar.gz *

config.json
merges.txt
pytorch_model.bin
special_tokens_map.json
tokenizer_config.json
tokenizer.json
vocab.json


In [25]:
fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, "model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)
print(os.path.join(bucket, key))

ai-inference-env/summarizer/model.tar.gz
